<a href="https://colab.research.google.com/github/Method-for-Software-System-Development/Cloud_Computing/blob/develop/logic/user_controller.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
# Navigate to project folder and ensure we're on the 'develop' branch
import os, subprocess, sys

REPO_DIR = "/content/Cloud_Computing"
if not os.path.isdir(REPO_DIR):
    subprocess.run([
        "git", "clone", "-b", "develop",
        "https://github.com/Method-for-Software-System-Development/Cloud_Computing.git",
        REPO_DIR
    ], check=True)

# Update & checkout develop if repo already exists
subprocess.run(["git", "-C", REPO_DIR, "fetch", "origin"], check=True)
subprocess.run(["git", "-C", REPO_DIR, "checkout", "develop"], check=True)
subprocess.run(["git", "-C", REPO_DIR, "pull"], check=True)

# Install importnb (only needed once per session)
%pip install importnb --quiet

# Add firebase folder to Python path and import its notebook as module
sys.path.append(os.path.join(REPO_DIR, "firebase"))
from importnb import Notebook

with Notebook():
    import FireBase as fb   # now fb.add_user, fb.get_user, etc. are available


In [7]:
def login(username: str, password: str) -> bool:
    """
    Authenticate a user by:
      1. Direct key lookup: /users/{username}
      2. Fallback: scan all users for matching 'username' field (snake_case)

    Args:
      username (str): the username key or the 'username' field value
      password (str): the password to check

    Returns:
      bool: True if credentials are valid, False otherwise
    """
    # 1) direct lookup by key
    status, user = fb.get_user(username)
    if status == 200 and user and user.get("password") == password:
        return True

    # 2) fallback: scan all users
    status, all_users = fb.get_all_users()
    if status != 200 or all_users is None:
        return False

    for u in all_users.values():
        if u.get("username") == username and u.get("password") == password:
            return True

    return False



In [9]:
from importnb import Notebook

with Notebook():
    import FireBase as fb

def login(username: str, password: str) -> bool:
    """
    Authenticate by key OR by 'username' field (snake_case).
    """
    # 1) direct key lookup
    st, user = fb.get_user(username)
    if st == 200 and user and user.get("password") == password:
        return True

    # 2) search all users by 'username' field
    st, all_users = fb.get_all_users()
    if st != 200 or all_users is None:
        return False
    for u in all_users.values():
        if u.get("username") == username and u.get("password") == password:
            return True
    return False
